In [ ]:
# load basics library
import pandas as pd
import numpy as np
import calendar
import xgboost as xgb
import numpy
from matplotlib import pyplot
from sklearn.cluster import MiniBatchKMeans
from tpot import TPOTRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from xgboost import XGBClassifier
from sklearn.svm.libsvm import cross_validation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.multiclass import type_of_target
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
'''use this model_selection rather than grid-server,
#  https://stackovlem o ferflow.com/questions/40257492/gridsearchcv-typeerror-stratifiedkfold-object-is-not-iterable?rq=1
#  can solve problem of “'StratifiedKFold' object is not iterable”
'''
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.layers import Dropout
#import commond.ipynb from same folder
import import_ipynb
from common_func import *

In [ ]:
X_train, y_train, X_test, y_test = label_data_asArray()

In [ ]:
#Todo: calculate the 1% sample from  all dataset

In [ ]:
length = y_test.shape[0]
X_train = X_train[:length, :]
y_train = y_train[:length]

In [ ]:
scalarX, scalarY = MinMaxScaler(), MinMaxScaler()
scalarX.fit(X_train)
scalarY.fit(y_train.reshape(length,1))
X = scalarX.transform(X_train)
y = scalarY.transform(y_train.reshape(length,1))

In [ ]:
y.shape

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimizer='rmsprop', init='glorot_uniform'):
    # create model
    model = Sequential()
    model.add(Dense(12, input_dim=X_train.shape[1], kernel_initializer=init, activation='relu'))
    model.add(Dense(8, kernel_initializer=init, activation='relu'))
    model.add(Dense(1, kernel_initializer=init, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [ ]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [ ]:
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
print(model.model.summary())

In [ ]:
# grid search epochs, batch size and optimizer
optimizers = ['rmsprop', 'adam']
init = ['glorot_uniform', 'normal', 'uniform']
epochs = [5, 10, 15]
batches = [5, 10, 20]
param_grid = dict(optimizer=optimizers, epochs=epochs, batch_size=batches, init=init)
grid = GridSearchCV(estimator=model, param_grid=param_grid)
grid_result = grid.fit(X_train, y_train)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

In [ ]:
length = y_test.shape[0]
X_test = X_test[:length, :]
y_test = y_test[:length]

scalarX.fit(X_test)
scalarY.fit(y_test.reshape(length,1))
X_test = scalarX.transform(X_test)
y_test = scalarY.transform(y_test.reshape(length,1))